In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV
import matplotlib.pyplot as plt
import datetime

# Loading data

In [3]:
weather_a = pd.read_csv('data/2149507.csv')
mobility = pd.read_csv('data/Global_Mobility_Report.csv', dtype = {'sub_region_1': str, 'sub_region_2': str})
china_mobility = pd.read_csv('data/China_(City Movement Intensity) 0101-0502.csv')
us_cases = pd.read_csv('data/US_covid_by_county.csv')
china_cases = pd.read_csv('data/China_covid_by_city.csv') 
decentralization = pd.read_excel('data/Kopie+von+Ivanyna+Shah+Decentralization+Dataset+Jan+2014.xls', 'Data')
mobilityus = pd.read_csv('data/mobilityus.csv')
covid_stringency = pd.read_csv('data/covid-stringency-index.csv')
county_pres = pd.read_csv('data/countypres_2000-2016.csv')
china_data_raw = pd.read_csv('data/chnmobility.csv')
china_pop_density = pd.read_csv('data/Chnpopden.csv')
age = pd.read_csv('data/election-context-2018.csv')
covidus = pd.read_csv('data/us-counties.csv')

##  United States: Centralized Dataframe

In [12]:
covidus = covidus.dropna().copy()

In [13]:
covidus['fips'] = ['0' + str(int(f)) if (len(str(int(f))) < 5) else str(int(f)) for f in covidus.fips]

In [17]:
covidus[covidus.date == '2020-03-15'].to_csv('data/us-counties-covid-0315.csv')

In [18]:
covidus[covidus.date == '2020-06-01'].to_csv('data/us-counties-covid-0601.csv')

In [19]:
age['county'] = [c + ' County' for c in age['county']]
age_columns = ['fips', 'state', 'county', 'white_pct', 'black_pct', 'hispanic_pct', 'nonwhite_pct', 'foreignborn_pct', 
               'female_pct', 'age29andunder_pct','age65andolder_pct', 'median_hh_inc', 'clf_unemploy_pct', 
               'lesshs_pct', 'lesscollege_pct', 'lesshs_whites_pct', 'lesscollege_whites_pct',
               'rural_pct', 'ruralurban_cc']

In [20]:
data = mobilityus.copy() #mobilityus.dropna().copy()
data.date = pd.to_datetime(data.date)
data['avg_mobility'] = data.filter(['retail', 'grocery', 'parks', 'transit', 'workplaces', 'residential']).mean(axis = 1)
data = data.merge(age[age_columns], on = ['state', 'county'], how = 'inner')


# adding interaction terms
data['demxSAH'] = data['dem_share'] * data['stay_at_home']
data['demxcases'] = data['dem_share'] * data['covid_cases']
data['popxpct_pov'] = data['pop'] * data['pct_pov']
data['SAHxpop'] = data['pop'] * data['stay_at_home']

data[data.state== 'Washington'].county.unique()

array(['Adams County', 'Asotin County', 'Benton County', 'Chelan County',
       'Clallam County', 'Clark County', 'Cowlitz County',
       'Douglas County', 'Ferry County', 'Franklin County',
       'Grant County', 'Grays Harbor County', 'Island County',
       'Jefferson County', 'King County', 'Kitsap County',
       'Kittitas County', 'Klickitat County', 'Lewis County',
       'Lincoln County', 'Mason County', 'Okanogan County',
       'Pacific County', 'Pend Oreille County', 'Pierce County',
       'San Juan County', 'Skagit County', 'Skamania County',
       'Snohomish County', 'Spokane County', 'Stevens County',
       'Thurston County', 'Walla Walla County', 'Whatcom County',
       'Whitman County', 'Yakima County'], dtype=object)

In [25]:
data[data['date'] >= '2020-05-20']['covid_cases']

95        127.0
96        136.0
97        147.0
98        149.0
99          NaN
          ...  
229958      NaN
230028      0.0
230029      0.0
230030      0.0
230031      NaN
Name: covid_cases, Length: 13262, dtype: float64

In [ ]:
data[data.state == 'California'].dem_share

In [ ]:
data.to_csv('data/usa_combined_data.csv')

## Plots
5 counties for average mobility across all sectors

In [ ]:
plt.figure(figsize = (10, 10))
for county in ['New York County', 'Cook County', 'Los Angeles County']:
    plt.plot(data[data.county == county].date, data[data.county == county].covid_cases, label = county)
plt.xticks(rotation = 45)
plt.legend()
plt.title('Coronavirus Cases in 3 US Counties')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
for county in ['New York County', 'Cook County', 'Los Angeles County']:
    plt.plot(data[data.county == county].date, data[data.county == county].avg_mobility, label = county)
plt.xticks(rotation = 45)
plt.legend()
plt.title('Average Mobility Difference in 3 US Counties')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
state = data[data.state == 'Illinois']
for column in ['retail', 'grocery', 'parks', 'transit', 'workplaces', 'residential']:
    plt.plot(state[state.county == 'Cook County'].date, state[state.county == 'Cook County'][column], label = column)
plt.xticks(rotation = 45)
plt.title('Cook County Mobility')
plt.legend()
plt.show()

## China Mobility Dataframe
#### compute baseline: the median value, for the corresponding day of the week, from Jan 1 - Jan-15


In [26]:
china = china_mobility.T.copy()
china.columns = china.iloc[0]
china = china[1:]
china.index = pd.to_datetime(china.index)
china = china.astype('float')
china = china.reset_index()
# find day of the week
china['weekday'] = china['index'].dt.dayofweek
temp = china[(china['index'] >= '2020-01-01') & (china['index'] <= '2020-01-15')]

baseline = temp.iloc[:, 1:].groupby('weekday').median()

# compute the percentage differences from the baseline
def compute_diff(row, baseline):
    day = row['weekday']
    return (row[1:-1] - baseline.loc[day, :]) * 100

modified_china = china.apply(lambda row: compute_diff(row, baseline), axis = 1)

modified_china['date'] = china['index']

In [27]:
# changing city names
city_names = []
for name in modified_china.columns[:-1]:
    city_names.append(name.split('shi_', 1)[0])
for i in china_data_raw.City.unique():
    if i not in city_names:
        print(i)
baseline.columns = city_names
baseline

,Beijing,Tianjin,Shijiazhuang,Tangshan,Qinhuangdao,Handan,Xingtai,Baoding,Zhangjiakou,Chengde,...,Kashidiqu_653100,Hetiandiqu_653200,Yilihasakezizhizhou_654000,Tachengdiqu_654200,Aleitaidiqu_654300,Shihezi,Alaer,Tumushuke,Wujiaqu,Tiemenguan
weekday,,,,,,,,,,,,,,,,,,,,,
0,6.39020,6.02305,5.50990,5.61520,5.36940,5.19520,4.91420,4.81560,4.80190,4.79325,...,5.79535,4.75840,4.66105,3.69495,3.47010,4.41800,3.86940,4.93400,3.22600,2.46425
1,6.42215,6.05810,5.70050,5.70775,5.50995,5.02605,4.83695,4.99225,4.95570,4.99075,...,5.51170,4.69075,4.58735,3.61315,3.42515,4.37600,3.70355,4.77665,3.16535,2.16795
2,6.37160,5.92800,5.76810,5.74870,5.53580,5.00250,4.90880,5.09970,4.95800,5.09620,...,5.62120,4.70770,4.62450,3.65500,3.44260,4.20930,3.86820,4.84880,3.20380,1.89110
3,6.53360,6.25470,5.82820,5.78705,5.65980,5.23110,5.09290,5.10605,5.06235,5.05170,...,5.75705,4.82040,4.66315,3.64995,3.44115,4.49570,4.08890,4.84100,3.27885,2.14925
4,6.62560,6.32105,5.92250,5.84080,5.72520,5.38970,5.23660,5.18480,5.19325,5.10560,...,5.80675,4.89535,4.69935,3.71790,3.51535,4.49670,3.94850,4.92175,3.27395,2.26540
5,4.92890,5.31215,5.31565,5.32365,5.11075,5.11145,5.00885,4.88105,4.77010,4.67560,...,5.68790,4.79850,4.44835,3.48970,3.29000,4.06050,3.56615,4.54365,3.05175,1.84395
6,4.65010,5.03415,4.82430,5.21970,4.92100,4.71215,4.62310,4.66640,4.62720,4.55605,...,5.51580,4.72050,4.38640,3.44930,3.25715,3.90485,3.52545,4.41980,2.95950,1.74845


## Centralized China Policy, Cases, and Mobility Dataframe

In [28]:
china_data = china_data_raw.drop('Unnamed: 0', axis = 1)
china_data['Date'] = pd.to_datetime(china_data['Date'])
china_data['Start.Date']= pd.to_datetime(china_data['Start.Date'])
china_data['weekday'] = china_data['Date'].dt.dayofweek
# compute the percentage differences from the baseline, for this new dataframe
def compute_diff_2(row, baseline):
    day = row['weekday']
    city = row['City']
    if city in ['Fuzhou', 'Suzhou', 'Taizhou']:
        return row['CMI'] - baseline.loc[day, city][0]
    return (row['CMI'] - baseline.loc[day, city]) * 1.0

china_data['mobility'] = pd.to_numeric(china_data.apply(lambda row: compute_diff_2(row, baseline), axis = 1, raw = True))
china_data['stay_at_home'] = (china_data.Date.values >= china_data['Start.Date'].values)
china_data['stay_at_home'] = china_data['stay_at_home'].replace({True: 1, False:0})

In [ ]:
china_data['City'].unique()

In [29]:
china_data = china_data.merge(china_pop_density[['Pop_den', 'City']], on = 'City')
china_data['SAHxA'] = china_data['Policy.A..Complete.Shutdown.'] * china_data['stay_at_home']
china_data['SAHxB'] = china_data['Policy.B..Partial.Shutdowns.'] * china_data['stay_at_home']
china_data['SAHxC'] = china_data['Policy.C..Checkpoints.and.Quarantine.Zones.'] * china_data['stay_at_home']
china_data['SAHxConfCount'] = china_data['stay_at_home'] * china_data['ConfCount']
china_data

,City,Date,CMI,Province,Start.Date,Policy.A..Complete.Shutdown.,Policy.B..Partial.Shutdowns.,Policy.C..Checkpoints.and.Quarantine.Zones.,population,ConfCount,weekday,mobility,stay_at_home,Pop_den,SAHxA,SAHxB,SAHxC,SAHxConfCount
0,Beijing,2020-01-22,3.9546,Beijing,2020-02-10,0,0,1,11716620,10,2,-2.41700,0,714.20,0,0,0,0
1,Beijing,2020-01-23,3.0974,Beijing,2020-02-10,0,0,1,11716620,22,3,-3.43620,0,714.20,0,0,0,0
2,Beijing,2020-01-24,2.4029,Beijing,2020-02-10,0,0,1,11716620,36,4,-4.22270,0,714.20,0,0,0,0
3,Beijing,2020-01-25,1.8254,Beijing,2020-02-10,0,0,1,11716620,41,5,-3.10350,0,714.20,0,0,0,0
4,Beijing,2020-01-26,1.6544,Beijing,2020-02-10,0,0,1,11716620,68,6,-2.99570,0,714.20,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,Zhumadian,2020-03-28,4.5740,Henan,2020-02-04,0,1,0,720000,139,5,-0.63685,1,605.72,0,1,0,139
5227,Zhumadian,2020-03-29,3.7015,Henan,2020-02-04,0,1,0,720000,139,6,-1.26230,1,605.72,0,1,0,139
5228,Zhumadian,2020-04-06,4.7071,Henan,2020-02-04,0,1,0,720000,139,0,-0.64680,1,605.72,0,1,0,139
5229,Zhumadian,2020-04-12,4.7413,Henan,2020-02-04,0,1,0,720000,139,6,-0.22250,1,605.72,0,1,0,139


In [36]:
china_data.Province.unique()

array(['Beijing', 'Sichuan', 'Guangdong', 'Hubei', 'Fujian', 'Guizhou',
       'Zhejiang', 'Shanxi', 'Anhui', 'Shandong', 'Jiangxi', 'Yunnan',
       'Gansu', 'Jiangsu', 'Guangxi', 'Hainan', 'Shanghai', 'Hebei',
       'Tianjin', 'Henan'], dtype=object)

In [48]:
china_data[china_data.Date == '2020-02-01'].Province.unique()

array(['Beijing', 'Sichuan', 'Guangdong', 'Hubei', 'Fujian', 'Zhejiang',
       'Shanxi', 'Anhui', 'Shandong', 'Jiangxi', 'Yunnan', 'Gansu',
       'Jiangsu', 'Guangxi', 'Hainan', 'Shanghai', 'Hebei', 'Tianjin',
       'Henan'], dtype=object)

In [40]:
china_data[china_data.Date == '2020-02-01'][['Province', 'ConfCount']].groupby('Province').sum().to_csv('data/plot_china_cases_0201.csv')

In [ ]:
china_data.to_csv('data/china_combined_data.csv')

In [ ]:
modified_china.columns

In [ ]:
plt.figure(figsize = (10, 10))
for city in ['Beijing', 'Shanghai', 'Wuhan']:
    for shi in modified_china.columns:
        if city in shi:
            plt.plot(modified_china.date, modified_china[shi], label = shi)
plt.xticks(rotation = 45)
plt.title('Mobility Difference in 3 Chinese Cities')
plt.legend()
plt.show()

## Double LASSO - Feature Selection

Determine an adequate regularization parameter for both LASSOs


In [ ]:
lambdas = np.logspace(-4, -1, 30)

In [ ]:
def double_lasso(data, treatment, covariates, lambdas, us = True):
    '''
    Treatment and covariates do not need to be standardized
    '''
    print('Treatment Variable: {} \nControl Variables: {}'.format(treatment, str(covariates)))
    print()
    X = data.filter([treatment] + covariates).values
    D = data.filter(covariates).values
    T = data[treatment].values
    responses = ['retail', 'grocery', 'parks', 'transit', 'workplaces', 'residential', 'avg_mobility'] if us else ['mobility']
    useful_controls = {}
    
    for response in responses:
        useful_controls[response] = covariates.copy()
        Y = data[response].values.astype('float')
        lasso1 = LassoCV(alphas = lambdas, normalize = True)
        lasso2 = LassoCV(alphas = lambdas, normalize = True)
        lasso1.fit(X, Y)
        lasso2.fit(D, T)
        print(response)
        beta_1 = lasso1.coef_
        beta_2 = lasso2.coef_
        print('LASSO 1 of Y on T and D: ' + str(beta_1))
        print('LASSO 2 of T on D: ' + str(beta_2))
        beta_1 = np.delete(beta_1, 0)
        print('Dropped Control Variables:')
        for i in range(len(beta_1)):
            if (beta_1[i] == 0) & (beta_2[i] == 0):
                print(covariates[i])
                useful_controls[response].remove(covariates[i])
        print()
    return useful_controls

In [ ]:
usa_controls = double_lasso(data, 'stay_at_home', 
             ['dem_share', 
              'income',
              'pop', 
              'pct_pov', 
              'covid_cases', 'demxSAH', 'demxcases', 'popxpct_pov','SAHxpop',
             'white_pct',
       'black_pct', 'hispanic_pct', 'nonwhite_pct', 'foreignborn_pct',
       'female_pct', 'age29andunder_pct', 'age65andolder_pct', 'median_hh_inc',
       'clf_unemploy_pct', 'lesshs_pct', 'lesscollege_pct',
       'lesshs_whites_pct', 'lesscollege_whites_pct', 'rural_pct',
       'ruralurban_cc'],
            lambdas)

In [ ]:
usa_controls['residential']

In [ ]:
usa_controls['workplaces']

In [ ]:
usa_controls['avg_mobility']

### For China

In [ ]:
double_lasso(china_data, 'stay_at_home', ['Policy.A..Complete.Shutdown.', 'Policy.B..Partial.Shutdowns.', 
                                         'Policy.C..Checkpoints.and.Quarantine.Zones.', 'population',
                                         'ConfCount', 'SAHxA', 'SAHxB', 'SAHxC', 'SAHxConfCount', 'Pop_den'], 
             lambdas, us = False)



### USA Double LASSO only with Dates after 3/15

In [ ]:
double_lasso(data[data.date > '2020-03-15'], 'stay_at_home', 
             ['dem_share', 
              'income',
              'pop', 
              'pct_pov', 
              'covid_cases', 'demxSAH', 'demxcases', 'popxpct_pov','SAHxpop'],
            lambdas)